In [2]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [3]:
# Load Data
data_dir = './data/'
class_names = ["apple", "bat", "circle", "clock", "cloud",
               "crown", "diamond", "donut", "fish",
               "hot_dog", "lightning", "mountain", "skull",
               "smiley_face", "square", "star", "sun", "t-shirt", "tree"]

In [4]:
X, y = [], []

for label, class_name in enumerate(class_names):
    file_path = os.path.join(data_dir, f"{class_name}.npy")
    data = np.load(file_path)  # shape: (30000, 784)
    X.append(data)
    y.append(np.full((data.shape[0],), label))

In [5]:

X = np.vstack(X)  # shape: (570000, 784)
y = np.hstack(y)  # shape: (570000,)

In [6]:
# ✅ Proper Normalization for image pixel data
X = X.astype('float32') / 255.0

In [7]:
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size=0.5, random_state=42, stratify=y_)

In [8]:
# ✅ Improved and Simplified Model
model = Sequential([
    Dense(512, activation='relu', input_shape=(784,), kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    Dropout(0.2),
    Dense(19, activation='linear')  # logits
])

In [9]:
# Compile
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [10]:

# ✅ Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [11]:

# ✅ Increased batch size for stability
model.fit(
    X_train, y_train,
    validation_data=(X_cv, y_cv),
    epochs=100,
    batch_size=512,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/100
891/891 - 42s - loss: 0.9493 - accuracy: 0.7603 - val_loss: 0.6256 - val_accuracy: 0.8565 - lr: 0.0010 - 42s/epoch - 47ms/step
Epoch 2/100
891/891 - 30s - loss: 0.6712 - accuracy: 0.8447 - val_loss: 0.5427 - val_accuracy: 0.8804 - lr: 0.0010 - 30s/epoch - 34ms/step
Epoch 3/100
891/891 - 32s - loss: 0.6044 - accuracy: 0.8630 - val_loss: 0.5103 - val_accuracy: 0.8898 - lr: 0.0010 - 32s/epoch - 35ms/step
Epoch 4/100
891/891 - 29s - loss: 0.5677 - accuracy: 0.8734 - val_loss: 0.4924 - val_accuracy: 0.8945 - lr: 0.0010 - 29s/epoch - 32ms/step
Epoch 5/100
891/891 - 28s - loss: 0.5451 - accuracy: 0.8795 - val_loss: 0.4755 - val_accuracy: 0.8988 - lr: 0.0010 - 28s/epoch - 31ms/step
Epoch 6/100
891/891 - 48s - loss: 0.5304 - accuracy: 0.8835 - val_loss: 0.4682 - val_accuracy: 0.9013 - lr: 0.0010 - 48s/epoch - 54ms/step
Epoch 7/100
891/891 - 33s - loss: 0.5196 - accuracy: 0.8865 - val_loss: 0.4578 - val_accuracy: 0.9045 - lr: 0.0010 - 33s/epoch - 37ms/step
Epoch 8/100
891/891 - 30s -

In [12]:
# Evaluation
logits_cv = model.predict(X_cv)
y_cv_pred = tf.argmax(tf.nn.softmax(logits_cv, axis=1), axis=1).numpy()
print("Validation Accuracy:", accuracy_score(y_cv, y_cv_pred))

logits_test = model.predict(X_test)
y_test_pred = tf.argmax(tf.nn.softmax(logits_test, axis=1), axis=1).numpy()
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

1782/1782 [==============================] - 6s 3ms/step
Validation Accuracy: 0.9293859649122806
1782/1782 [==============================] - 12s 7ms/step
Test Accuracy: 0.9311578947368421


In [14]:
# Save model and normalization
model.save("doodle_model.keras")